In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##import library

In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import cv2
from tensorflow import keras
from keras import layers
import os
from keras.preprocessing import image
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

##load dataset

In [52]:
from logging import info
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
train ,validation ,test = tfds.load('cats_vs_dogs',split=['train[:70%]','train[70%:80%]','train[80%:]'], shuffle_files=True, as_supervised=True)

##data preprocessing

In [53]:
def normalization(img, label):
    temp = (tf.image.resize(img, (output_dim, output_dim))) / 255
    return temp
output_dim = 56
train = train.map(normalization)

In [54]:
list_train = list(train.as_numpy_iterator())
train = np.array(list_train)

In [ ]:
train.shape

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1) 
  plt.imshow(train[i])
  plt.xticks([])
  plt.yticks([])
plt.show()

In [ ]:
print(np.max(train[:,:,:]), np.min(train[:,:,:]))

##DCGAN

make generator

In [ ]:
latent_dim = 56
height = 56
width = 56
channels = 3
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 28 * 28)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((28, 28, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()
'''latent_dim = 56
height = 56
width = 56
channels = 3
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 14 * 14)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14, 14, 64))(x)
x = layers.Conv2D(64, 5,strides=1, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(128, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()'''


make discriminator

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = keras.optimizers.RMSprop( lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [ ]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

In [71]:
iterations = 3000
batch_size = 20
save_dir = '/content/drive/MyDrive/logs/gan'

In [72]:
real_images = train[0: 20]
random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
random_latent_vectors.shape
generated_images = generator.predict(random_latent_vectors)
generated_images.shape  , real_images.shape

((20, 56, 56, 3), (20, 56, 56, 3))

In [ ]:
start = 0
noise_strenght = 0.1
for step in range(iterations):
  #تولید بیست بردار رندوم با بعد 56
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  #تولید بیست تصویر با استفاده از مولد
  generated_images = generator.predict(random_latent_vectors)

  stop = start + batch_size
  #بیست تصویر واقعی 
  real_images = train[start: stop]
  real_images += noise_strenght * np.random.random(real_images.shape)
  #ترکیب تصاویر واقعی و مجازی
  combined_images = np.concatenate([generated_images, real_images])
  #اختصاص برچسب یک به مجازی و صفر به واقعی    
  labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
  #یه تریک برای اموزش
  labels += 0.05 * np.random.random(labels.shape)
  #اموزش متمایزگز
  d_loss = discriminator.train_on_batch(combined_images, labels)
  #تولید بیست نمونه مجازی
  random_latent_vectors = np.random.normal(size=( batch_size, latent_dim))
  #به بیست نمونه مجازی برچسب واقعی میدهیم
  misleading_targets = np.zeros((batch_size, 1))
  #اموزش مولد
  a_loss = gan.train_on_batch( random_latent_vectors, misleading_targets)
  start += batch_size
  if start > len(train) - batch_size:
    start = 0
  if step % 100 == 0:
    gan.save_weights('gan.h5')
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)
    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir , 'generated' + str(step) + '.png'))
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, 'real' + str(step) + '.png'))

genearte 100 sample from dcgan

In [ ]:
noise = np.random.normal(size=(100, latent_dim))
show_images(noise,(10,10))

In [32]:
def show_images(noise, size_fig):
    generated_images = generator.predict(noise)   #Create the images from the GAN.
    plt.figure(figsize=size_fig)
    
    for i, image in enumerate(generated_images):
        plt.subplot(size_fig[0], size_fig[1], i+1)
        plt.imshow(image.reshape((56, 56, 3)))
        plt.axis('off')
    
    plt.tight_layout()   #Tight layout so that all of the generated images form a nice grid
    plt.show()

##FCGAN

In [ ]:
optimizer = Adam(0.0002, 0.5)
def create_generator():
    generator = Sequential()

    generator.add(Dense(128, input_dim=56))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(256))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(2048, input_dim=56))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(56*56*3, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator  

In [ ]:
fc_generated = create_generator()
fc_generated.summary()

In [ ]:
def create_discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(1024, input_dim=56*56*3))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer = optimizer)
    return discriminator

In [ ]:
fc_discriminator = create_discriminator()
fc_discriminator.summary()

In [ ]:
fc_discriminator.trainable = False 
fc_gan_input = keras.layers.Input(shape=(56,))
fc_fake_image = fc_generated(fc_gan_input)
fc_gan_output = fc_discriminator(fc_fake_image)
fc_gan =keras.models.Model(fc_gan_input, fc_gan_output)
fc_gan.compile(loss='binary_crossentropy', optimizer=optimizer)
fc_gan.summary()

train the model

In [ ]:
train_x = train[:16000]
train_x.shape

(16000, 56, 56, 3)

In [ ]:
latent_dim

56

In [ ]:
latent_dim = 56
batch_size = 20
random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
generated_images = fc_generated.predict(random_latent_vectors)
real_images = train[0: 20]
new = np.reshape(real_images,(20, 9408))
generated_images.shape , new.shape

((20, 9408), (20, 9408))

In [ ]:
type(real_images)

numpy.ndarray

In [ ]:
generated_images.shape

(20, 9408)

In [ ]:
start = 0
iterations = 1000
batch_size = 20 
save_directory = '/content/drive/MyDrive/logs/fc_gan'
discriminator_loss = []
genarator_loss = []
noise_strenght = 0.003
for step in range(iterations):
  #تولید بیست بردار رندوم با بعد 56
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  #تولید بیست تصویر با استفاده از مولد
  generated_images = fc_generated.predict(random_latent_vectors)

  stop = start + batch_size
  #بیست تصویر واقعی 
  real_images = train[start: stop]
  #اضافه کردن نویز به تصاویر ورودی برای متعادل سازی
  real_images += noise_strenght * np.random.random(real_images.shape)
  new_real_images = np.reshape(real_images,(20, 9408))
  #ترکیب تصاویر واقعی و مجازی
  combined_images = np.concatenate([generated_images, new_real_images])
  #اختصاص برچسب یک به مجازی و صفر به واقعی    
  labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
  #یه تریک برای اموزش
  labels += 0.05 * np.random.random(labels.shape)
  #اموزش متمایزگز
  d_loss = fc_discriminator.train_on_batch(combined_images, labels)
  #تولید بیست نمونه مجازی
  random_latent_vectors = np.random.normal(size=( batch_size, latent_dim))
  #به بیست نمونه مجازی برچسب واقعی میدهیم
  misleading_targets = np.zeros((batch_size, 1))
  #اموزش مولد
  a_loss = fc_gan.train_on_batch( random_latent_vectors, misleading_targets)
  start += batch_size
  discriminator_loss.append(d_loss)
  genarator_loss.append(a_loss)

  if start > len(train) - batch_size:
    start = 0
  if step % 100 == 0:
    fc_gan.save_weights('gan.h5')
    print('discriminator loss:', d_loss)
    print('generator loss:', a_loss)

    generated_images = np.reshape(generated_images,(20,56,56,3))

    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_directory , 'fc_gan_generated' + str(step) + '.png'))
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_directory, 'fc_real' + str(step) + '.png'))

In [ ]:
plt.plot(discriminator_loss, label='discriminator')
plt.plot(genarator_loss, label='genarator')
plt.title('LOSS')
plt.legend()
plt.show()